In [107]:
!curl ipecho.net/plain

35.237.15.45

In [108]:
!pip install redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
import pandas as pd
import psycopg2
import redis
import numpy as np

In [110]:
# Redis Cloud Instance Information
redis_host = 'redis-14444.c1.asia-northeast1-1.gce.cloud.redislabs.com'
redis_port = 14444
redis_password = 'KsxcueiwoTi92w0evIzbvdpf34ZUqmWx'

In [111]:
# Postgres Database Information
pg_host = '35.223.9.251'
pg_database = 'kamulu4_DB'
pg_user = 'kamulu4'
pg_password = 'test'

In [112]:
# Redis Client Object
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)




In [ ]:
def transform_data(data):
    # Drop any rows with missing data
    data = data.dropna()

    # Convert the call date to a datetime object
    data['call_date'] = pd.to_datetime(data['call_date'])

    # Convert the call cost column to a numeric type
    data['call_cost'] = pd.to_numeric(data['call_cost'], errors='coerce')

    # Calculate the call cost in USD
    data['call_cost_usd'] = data['call_cost'] * 0.8



    # Rename columns
    data = data.rename(columns={'id': 'customer_id', 'destination': 'call_destination', 'call_duration': 'call_duration_min'})
    

    return data


def load_data(transformed_data):
    # Connect to Postgres database
    conn = psycopg2.connect(host=pg_host,database=pg_database, user=pg_user, password=pg_password)

    # Create a cursor object
    cur = conn.cursor()

    # Create a table to store the data
    cur.execute('CREATE TABLE IF NOT EXISTS customer_call_logs (\
                 customer_id INT,\
                 call_cost_usd FLOAT,\
                 call_destination VARCHAR,\
                 call_date TIMESTAMP,\
                 call_duration_min FLOAT\
                 )')

    # Insert the transformed data into the database
    for i, row in transformed_data.iterrows():
        call_cost_usd = row['call_cost_usd']
        if pd.isna(call_cost_usd):
            call_cost_usd = 'NULL'
        else:
            call_cost_usd = float(call_cost_usd)
        call_duration_min = float(row['call_duration_min'].total_seconds() / 60)
        cur.execute(f"INSERT INTO customer_call_logs (customer_id, call_cost_usd, call_destination, call_date, call_duration_min) VALUES ({row['customer_id']}, {call_cost_usd}, '{row['call_destination']}', '{row['call_date']}', '{call_duration_min}')")

    # Commit the changes
    conn.commit()
    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()


def data_pipeline():
    # Data pipeline function
    data = extract_data()
    transformed_data = transform_data(data)
    load_data(transformed_data)

if __name__ == '__main__':
    # Run the data pipeline function
    data_pipeline()

